In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow. keras.models import Sequential
from tensorflow.keras import regularizers


# 체크포인트를 만드는 라이브러리 부분 임포트
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import cv2

In [66]:
#=============================
import random as rn

seed_num = 0
np.random.seed(seed_num)
rn.seed(seed_num)
tf.random.set_seed(seed_num)
#=============================

In [67]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [68]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

In [69]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [70]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
#   except RuntimeError as e:
#     # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
#     print(e)

In [71]:
with_helmet = os.listdir('./data/with_helmet_BW/')
without_helmet = os.listdir('./data/without_helmet_BW/')

In [72]:
with_y = [1] * len(with_helmet)
without_y = [0] * len(without_helmet)

y = with_y + without_y

In [73]:
len(y)

20000

In [74]:
image_data_label = [(cv2.imread('./data/with_helmet_BW/'+i),1) for i in with_helmet] + [(cv2.imread('./data/without_helmet_BW/'+ i),0) for i in without_helmet]

In [75]:
rn.shuffle(image_data_label)

In [76]:
x = [i[0] for i in image_data_label]
y = [i[1] for i in image_data_label]

In [77]:
# x = [cv2.imread('./data/with_helmet_resize/'+i) for i in with_helmet] + [cv2.imread('./data/without_helmet_resize/'+ i) for i in without_helmet]
# len(x)

In [78]:
x = np.array(x)
y = np.array(y)

In [80]:
x.shape

(20000, 112, 112, 3)

In [13]:
x = x.astype('float32') / 255

In [14]:
model = Sequential()

model.add(Conv2D(input_shape=(112,112,3), filters = 64, kernel_size = (3,3), activation = 'relu', padding='same'))

# 1층
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001))) # padding 데이터를 압축하는 옵션
model.add(BatchNormalization())

# 2층
model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001))) # padding 데이터를 압축하는 옵션
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2))) # strides 데이터를 압축하는 옵션
model.add(Dropout(0.2))

# 3층
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())

# 4층
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.3))

# 5층
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())

# 6층
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(units = 1, activation = 'sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 64)      36928     
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 64)     256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 64)       0

In [16]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x, y, validation_split=0.2, epochs = 100,  batch_size = 32, shuffle=True)

Epoch 1/100
500/500 [==============================] - 42s 75ms/step - loss: 0.6036 - accuracy: 0.8589 - val_loss: 2.1557 - val_accuracy: 0.7340
Epoch 2/100
500/500 [==============================] - 37s 74ms/step - loss: 0.2668 - accuracy: 0.9323 - val_loss: 72.5055 - val_accuracy: 0.1735
Epoch 3/100
500/500 [==============================] - 37s 74ms/step - loss: 0.3222 - accuracy: 0.9353 - val_loss: 35.7091 - val_accuracy: 0.2215
Epoch 4/100
500/500 [==============================] - 37s 74ms/step - loss: 0.1802 - accuracy: 0.9669 - val_loss: 52.6569 - val_accuracy: 0.4753
Epoch 5/100
500/500 [==============================] - 37s 74ms/step - loss: 0.1716 - accuracy: 0.9729 - val_loss: 87.5266 - val_accuracy: 0.2292
Epoch 6/100
500/500 [==============================] - 37s 74ms/step - loss: 0.1998 - accuracy: 0.9691 - val_loss: 50.5609 - val_accuracy: 0.2395
Epoch 7/100
500/500 [==============================] - 37s 74ms/step - loss: 0.1552 - accuracy: 0.9807 - val_loss: 177.3302 -

KeyboardInterrupt: 